In [ ]:
!pip install mido
!pip install pygame
!pip install music21
!pip install scikit-learn==1.3.0

In [1]:
# import some useful libraries
import glob, nltk, joblib
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import metrics

from PIL import Image
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer

from music21 import midi
from plugins.midi2img import midi2img
from plugins.img2midi import img2midi
from IPython.display import clear_output

# Load In museGAN dataset for visualization purposes
It turned out that the people at museGAN is leveraging midi -> image conversion. The image consisted of bar of a multi track piano roll. From the below image, the horizontal represent time and the vericle represent the instrument used. In this dataset the instrument are layered from bottom to top as piano, strings, guitar, drums, bass.


In [2]:
DATA_PATH = "../data/dataset/"

In [3]:
# download the punkt tokenizer from nltk to tokenize the piece caption
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ktrin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# load the training data
# read in the training data
training_set = joblib.load(f'{DATA_PATH}/training_set.joblib')
training_set[0]

# conver tuple to DF
training_set = pd.DataFrame(training_set, columns=['id', 'image', 'caption_list'])
training_set.head()

,id,image,caption_list
0,commu00001,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, main, melody, cinematic, string..."
1,commu00002,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, mid, low, accompaniment, newage, aco..."
2,commu00003,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, high, riff, cinematic, string, ..."
3,commu00004,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, mid, pad, cinematic, choir"
4,commu00005,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, low, pad, cinematic, acoustic, ..."


# Construction of museGAN
## External Data Source
For whatever reason, if we wanted to perform GAN modeling, we can leverage conversion of MIDI data to that of the piano roll. Download the data from piano repo in README and start performing the things below. [Convert-MIDI-TO-NP-ARRAY](https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c)

# Caption Processing

In [5]:
MAX_SEQ_LENGTH = 10 # 18 + start, end
EMBED_DIM = 100 
MAX_VOCAB_SIZE = 20000

In [6]:
# load in the metadata
# create a list of captions that concatenate the piece description and arousal
# lower case te caption list
# midi_meta = pd.read_csv(f'{DATA_PATH}/midi_meta.csv')
# midi_meta['caption_list'] = midi_meta['piece_description'].str.lower()+ ". " + midi_meta['piece_arousal'].str.lower()
# midi_meta

In [7]:
# build a vocabulary using sklearn count vectorizer to create a vocab from the most frequent words
input_captions = []
max_caption_length = -1 

for caption in tqdm(training_set['caption_list'].values):
    tokenized_caption = nltk.word_tokenize(caption, language='english')

    if len(tokenized_caption) > max_caption_length:
        max_caption_length = len(tokenized_caption)

    caption = (' '.join(tokenized_caption)).lower()
    input_captions.append(caption)


vectorizer = CountVectorizer(max_features=MAX_VOCAB_SIZE)
vectorizer.fit(input_captions)
vocab = vectorizer.get_feature_names_out()
MAX_VOCAB_SIZE = len(vocab)

  0%|          | 0/10246 [00:00<?, ?it/s]

In [8]:
# turn vocab into a dictionary of words and token id
# replace some words with special tokens like start/end/unk
# if the caption is too short, pad it with <pad> token
id_vocab_dict = {}
vocab_id_dict = {}

for sid, svocab in enumerate(vocab):
    id_vocab_dict[sid] = svocab
    vocab_id_dict[svocab] = sid

id_vocab_dict[MAX_VOCAB_SIZE] = "<unk>"
id_vocab_dict[MAX_VOCAB_SIZE + 1] = "<start>"
id_vocab_dict[MAX_VOCAB_SIZE + 2] = "<end>"
id_vocab_dict[MAX_VOCAB_SIZE + 3] = "<pad>"

vocab_id_dict["<unk>"] = MAX_VOCAB_SIZE
vocab_id_dict["<start>"] = MAX_VOCAB_SIZE + 1
vocab_id_dict["<end>"] = MAX_VOCAB_SIZE + 2
vocab_id_dict["<pad>"] = MAX_VOCAB_SIZE + 3

In [9]:
# tokenization - take the input caption and tokenize it
# declare a max sequence length 
def convert_text_to_data(texts, 
                         vocab_id_dict, 
                         max_length=20, 
                         type=None):
    """
        Function to convert text based data into tokenized data with proper padding
    """

    processed_data = []
    for text_num, text in enumerate(texts):
        sentence_ids = []

        # split the sentence into token
        # use the vocab to turn the word token into number
        for token in text.split():
            if token in vocab_id_dict.keys():
                sentence_ids.append(vocab_id_dict[token])
            else:
                sentence_ids.append(vocab_id_dict["<unk>"])

        vocab_size = len(vocab_id_dict.keys())

        # for decoder cases:
        # input sentence: <start>, [tokenize words from vocab], <end>, padded with <unk>
        # ouput sentence has: [tokenize words from vocab], <end>, padded with <unk>
        if type == 'input_target':
            ids = ([vocab_size - 3] + sentence_ids + [vocab_size - 2] + [vocab_size - 1] * max_length)[:max_length]
        elif type == 'output_target':
            ids = (sentence_ids + [vocab_size - 2] + [vocab_size - 1] * max_length)[:max_length]
        processed_data.append(ids)

    return np.array(processed_data)


train_target_input_data = convert_text_to_data(input_captions,
                                                vocab_id_dict,
                                                type='input_target',
                                                max_length=MAX_SEQ_LENGTH)
len(train_target_input_data)

10246

In [10]:
# added the tokenized caption to the metadata
training_set['tokenized_captions'] = train_target_input_data.tolist()
training_set

,id,image,caption_list,tokenized_captions
0,commu00001,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, main, melody, cinematic, string...","[57, 56, 56, 34, 56, 33, 56, 29, 56, 32]"
1,commu00002,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, mid, low, accompaniment, newage, aco...","[57, 56, 56, 30, 56, 33, 56, 28, 56, 5]"
2,commu00003,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, high, riff, cinematic, string, ...","[57, 56, 56, 34, 56, 33, 56, 26, 56, 42]"
3,commu00004,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, mid, pad, cinematic, choir","[57, 56, 56, 30, 56, 33, 56, 39, 56, 15]"
4,commu00005,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, low, pad, cinematic, acoustic, ...","[57, 56, 56, 34, 56, 33, 56, 28, 56, 39]"
...,...,...,...,...
10241,commu10377,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, low, bass, cinematic, electric, bass","[57, 56, 56, 30, 56, 28, 56, 8, 56, 15]"
10242,commu10378,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, pad, cinematic, string, ensembl...","[57, 56, 56, 34, 56, 33, 56, 39, 56, 15]"
10243,commu10379,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","a, minor, mid, pad, cinematic, string, ensemble","[57, 56, 56, 34, 56, 33, 56, 39, 56, 15]"
10244,commu10380,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","c, major, low, bass, cinematic, acoustic, piano","[57, 56, 56, 30, 56, 28, 56, 8, 56, 15]"


In [11]:
# create image, tokenized pair
# create image-caption pairs
datasets = []
for i, row in training_set.iterrows():
    caption = np.array(row['tokenized_captions'])
    images = np.array(row['image'])
    try:
        datasets.append((images, caption))
    except:
        pass

# GAN Definition

GAN model consists of two part:
1. Generator
2. Discriminator

In [12]:
# check to see if tensorflow mount to GPU properly
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
def caption_enhanced_generator(latent_dim=100, 
                               caption_dim=MAX_SEQ_LENGTH, 
                               vocab_size=len(vocab_id_dict.keys()), 
                               embed_dim=EMBED_DIM):
    """Define the generator model
        Inputs:
            latent_dim: dimension of the latent space
        Output:
            model: the generator model
    """
    n_nodes = 128 * 53 * 53

    # vectorized input layers
    input_layer = keras.layers.Input(shape=(latent_dim,), name='input_layer')
    
    # # vectorized caption input layers
    # # apply word embedding to the caption
    caption_input_layer = keras.layers.Input(shape=(caption_dim,), name='caption_input_layer')
    embedding_layer  = keras.layers.Embedding(input_dim=vocab_size,
                                                output_dim=embed_dim,
                                                name='caption_embedding_layer')
    embed_caption = embedding_layer(caption_input_layer)

    # # source_image_encoding = keras.layers.GlobalAveragePooling2D()(dense4)
    # # using LSTM to encode the caption with the input layer
    # lstm_layer = keras.layers.LSTM(100, return_sequences=True, return_state=True, name="decoder_lstm_layer")
    # decoder_output, decoder_state_h_output, decoder_state_c_output = lstm_layer(embed_caption, initial_state=[input_layer, input_layer])

    # pool the caption layer
    global_average_pooling1d_embed = keras.layers.GlobalAveragePooling1D()(embed_caption)

    # concat the input layer and the caption layer
    concat_layer = keras.layers.Concatenate(axis=1)([input_layer, global_average_pooling1d_embed])

    # apply 1D Global Average Pooling to the output of the dense layer on the caption decoded
    # global_average_pooling1d_layer = keras.layers.GlobalAveragePooling1D()(concat_layer)

    # Dense Layer 1
    dense1 = keras.layers.Dense(n_nodes)(concat_layer)
    leaky_relu1 = keras.layers.LeakyReLU(alpha=0.35)(dense1)
    reshape_layer = keras.layers.Reshape((53, 53, 128))(leaky_relu1)

    # Dense Layer 2
    dense2 =  keras.layers.Dense(1024)(reshape_layer)

    # Conv2DTranspose Layer
    conv2d_transpose = keras.layers.Conv2DTranspose(1024, (4, 4), strides=(2, 2), padding='same')(dense2)

    # Dense Layer 3
    dense3 =  keras.layers.Dense(1024)(conv2d_transpose)
    leaky_relu2 = keras.layers.LeakyReLU(alpha=0.35)(dense3)

    # Dense Layer 4
    dense4 =  keras.layers.Dense(512)(leaky_relu2)

    # Conv2D Layer
    conv2d = keras.layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')(dense4)

    # Create the model
    model = keras.Model(inputs=[input_layer,caption_input_layer], outputs=conv2d, name='generator')
    return model


In [14]:
def caption_enhanced_discriminator(in_shape = (106,106,1)):
    """
        GAN discriminator model
        Inputs:
            in_shape: shape of the input image
        Output:
            model: discriminator model with binary crossentropy loss to denotes if the image is real or fake
    """
    # Input Layer
    input_layer = keras.layers.Input(shape=in_shape, name='input_layer')
    
    # 2D Convlution Layer 1
    conv1 = keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same')(input_layer)
    leaky_relu1 = keras.layers.LeakyReLU(alpha=0.2)(conv1)
    dropout1 = keras.layers.Dropout(0.5)(leaky_relu1)

    # 2D Convlution Layer 2
    conv2 = keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same')(dropout1)
    leaky_relu2 = keras.layers.LeakyReLU(alpha=0.2)(conv2)
    dropout2 = keras.layers.Dropout(0.5)(leaky_relu2)

    # Flatten Layer
    flatten_layer = keras.layers.Flatten()(dropout2)

    # Batch Normalization Layer
    batch_normalization = keras.layers.BatchNormalization()(flatten_layer)

    # Dense Output Disminator Layer
    discriminate_layer = keras.layers.Dense(1, activation='sigmoid')(batch_normalization)

    # Create the model
    model = keras.Model(inputs=input_layer, outputs=discriminate_layer, name='discriminator_model')
    
    # model compile
    opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5) #3
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [15]:
def caption_enhanced_miniGAN(g_model, d_model,
                             g_model_input_shape=100, 
                             g_model_caption_input_shape=MAX_SEQ_LENGTH):
    """
        GAN model architecture
        Inputs:
            g_model: generator model
            d_model: discriminator model
            g_model_input_shape: shape of the input to the generator model
            g_model_caption_input_shape: shape of the input caption to the generator model
        Output:
            model: GAN model
    """
    # Pause the training of the discriminator
    d_model.trainable = False

    # Define the input layer for the generator
    generator_input = keras.layers.Input(shape=(g_model_input_shape))  # Specify the shape of the generator's input
    caption_input = keras.layers.Input(shape=(g_model_caption_input_shape))  # Specify the shape of the generator's input

    # Define the output of the generator
    generator_output = g_model([generator_input, caption_input])

    # Define the output of the discriminator
    discriminator_output = d_model(generator_output)

    # Create the model
    model = keras.Model(inputs=[generator_input, caption_input], outputs=discriminator_output)

    # Compile the model
    opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5, beta_2=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [16]:
def generate_real_samples(dataset, n_samples):
    # slice the image and caption from the dataset
    # generate 'real' class labels (1)
    images, captions = zip(*dataset)
    images = np.array(images)
    captions = np.array(captions)
    ix = np.random.randint(0, images.shape[0], n_samples)
    X_img = images[ix]   
    X_cap = captions[ix]
    y = np.ones((n_samples, 1))
    return X_img, X_cap, y
 
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(g_model, latent_dim, caption, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict([x_input, caption])
    y = np.zeros((n_samples, 1))
    return X, y

In [17]:
latent_dim = 100
g_model = caption_enhanced_generator(latent_dim)
d_model = caption_enhanced_discriminator()
gan_model = caption_enhanced_miniGAN(g_model, d_model)
g_model.summary(), d_model.summary(), gan_model.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input_layer (InputLayer [(None, 10)]         0                                            
__________________________________________________________________________________________________
caption_embedding_layer (Embedd (None, 10, 100)      6000        caption_input_layer[0][0]        
__________________________________________________________________________________________________
input_layer (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 100)          0           caption_embedding_layer[0][0]    
__________________________________________________________________________________________

c:\Users\ktrin\miniconda3\envs\W266\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


(None, None, None)

In [18]:
# plot the generated model
# keras.utils.plot_model(g_model, show_shapes=True, dpi=90)

In [19]:
# # plot the generated model
# keras.utils.plot_model(d_model, show_shapes=True, dpi=90)

In [20]:
# # plot the generated model
# keras.utils.plot_model(gan_model, show_shapes=True, dpi=90)

In [21]:
# # generate samples and save as a plot and save the model
# def summarize_performance(step, g_model, gan_model, latent_dim, x_cap, n_samples=100):
# 	# prepare fake examples
# 	X, _ = generate_fake_samples(g_model, latent_dim, x_cap, n_samples)
#     # save plot
# 	# scale from [-1,1] to [0,1]
# 	X = (X + 1) / 2.0
# 	# plot images
# 	for i in range(100):
# 		# define subplot
# 		plt.subplot(10, 10, 1 + i)
# 		# turn off axis
# 		plt.axis('off')
# 		# plot raw pixel data
# 		plt.imshow(X[i, :, :, 0], cmap='gray_r')
# 	# save plot to file
# 	filename1 = 'generated_plot_%04d.png' % (step+1)
# 	plt.savefig(filename1)
# 	plt.close()
# 	# save the generator model
# 	filename2 = 'model_%04d.h5' % (step+1)
# 	g_model.save(filename2)
# 	# save the gan model
# 	filename3 = 'gan_model_%04d.h5' % (step+1)
# 	gan_model.save(filename3)
# 	print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [22]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=16):
    bat_per_epo = int(len(dataset) / n_batch)
    half_batch = int(n_batch / 2)

    for i in range(n_epochs):
        for j in range(bat_per_epo):
            # prepare a mini batch of real and fake sample
            X_img_real, x_cap_real, y_real = generate_real_samples(dataset, half_batch)
            X_img_fake, y_fake = generate_fake_samples(g_model, latent_dim, x_cap_real, half_batch)

            print(np.unique(X_img_real), np.unique(X_img_fake))
            # print(X_img_real, X_img_fake)
            
            # use real caption input to generate a fake image
            # train the discriminator on fake images generate from real caption and a set of latent points
            X, y = np.vstack((X_img_real, X_img_fake)), np.vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)


            # print("Discriminator Loss: ", (d_loss, _))
            # prepare points in latent space and fetch some real caption input

            X_img_real_full, x_cap_real_full, y_real_full = generate_real_samples(dataset, n_batch)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))

            # perform full batch training on e2e GAN model 
            # using the latent point and real caption as input
            g_loss = gan_model.train_on_batch([X_gan, x_cap_real_full], y_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))

            # test print the image
            # X = g_model.predict([X_gan, x_cap_real_full])
            # print(np.unique(X))

        if (i+1) % 2 == 0:
            # summarize_performance(i, g_model, d_model, dataset, latent_dim)
            clear_output()


In [23]:
len(datasets)

10246

In [24]:
# read in the training data
import random
random.seed(544)

training_dataset = random.sample(datasets, 4096) #5 is the lenth of the sample
# training_dataset = datasets

In [25]:
train(g_model, d_model, gan_model, training_dataset, latent_dim, n_epochs=21, n_batch=18)

[0. 1.] [0.00000000e+00 1.20833462e-38 1.30470836e-38 2.05165495e-38
 2.05453715e-38 2.13292788e-38 2.75979259e-38 2.92883206e-38
 2.98979022e-38 5.03146359e-38 5.56337743e-38 5.56473557e-38
 5.58958507e-38 5.98925109e-38 6.68965313e-38 8.37612200e-38
 8.91075884e-38 8.91960048e-38 3.02059458e-37 3.48730970e-37
 4.52960917e-37 4.77037019e-37 5.22230576e-37 5.38331338e-37
 5.54822625e-37 5.60254552e-37 7.51034499e-37 1.21015328e-36
 1.27249434e-36 1.53206647e-36 1.61640787e-36 1.95136726e-36
 2.24642107e-36 3.14681179e-36 3.57970639e-36 3.67108880e-36
 4.34518072e-36 5.69666165e-36 5.70187942e-36 7.15656228e-36
 1.08457226e-35 1.11283621e-35 1.17044971e-35 1.27910890e-35
 1.29402872e-35 1.50738855e-35 1.51809887e-35 1.52088119e-35
 1.63495710e-35 2.52501725e-35 2.61664354e-35 2.61728265e-35
 2.97374500e-35 3.12483879e-35 3.26781460e-35 4.20281331e-35
 4.41197698e-35 4.67379108e-35 7.03119593e-35 1.78520697e-34
 4.45502073e-34 5.52658348e-34 6.86703826e-34 6.97156037e-34
 7.42254612e-34 

ResourceExhaustedError:  OOM when allocating tensor with shape[18,1024,106,106] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradient_tape/model/generator/leaky_re_lu_1/LeakyRelu/LeakyReluGrad-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_2642]

Function call stack:
train_function


In [46]:
# save the model
g_model.save('../models/mini-gan/comMU_generator_model_keyword_noLSTM_g20.h5')
d_model.save('../models/mini-gan/comMU_discriminator_model_keyword_noLSTM_g20.h5')
gan_model.save('../models/mini-gan/comMU_gan_model_keyword_noLSTM_g20.h5')

In [ ]:
g_model = keras.models.load_model('../models/mini-gan/comMU_generator_model_keyword_noLSTM_g20.h5')

In [48]:
# select a random row from the metadata to get the caption
row = training_set.sample(1, random_state=86)

# get a random image tokenize caption and actual caption
NLP_caption = row['caption_list'].values
caption = [np.array(a) for a in row['tokenized_captions'].values]
caption = np.array(caption)
NLP_caption, caption

(array(['a, minor, mid, low, accompaniment, cinematic, string, ensemble'],
       dtype=object),
 array([[57, 56, 56, 34, 56, 33, 56, 28, 56,  5]]))

In [49]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


# declare a latent space
latent_dim = 100
latent_points = generate_latent_points(latent_dim, 1)
latent_points.shape, caption.shape

((1, 100), (1, 10))

In [50]:
model = g_model
X = g_model.predict([latent_points, caption])#*400
array = np.array(X.reshape(106,106),dtype = np.uint8)
np.unique(array)

array([0, 1], dtype=uint8)

In [51]:
array*=255
new_image = Image.fromarray(array,'L')
new_image = new_image.save(f'{DATA_PATH}/images/captioned_piece_test_keyword_noLSTM.png')

In [52]:
# reconvert MIDI images to MIDI files
image_path = f'{DATA_PATH}/images/captioned_piece_test_keyword_noLSTM.png'
output_path = f'{DATA_PATH}/images/desc'

img2midi_obj = img2midi(image_path, output_path, resolution=0.25)
img2midi_obj.convert_to_midi(name="captioned_piece_test_keyword_noLSTM")

In [54]:
# run to cell to play
# stop the cell and run sp.stop to stop the music
from music21 import midi, converter, instrument, note, chord

mf = midi.MidiFile()
mf.open(f"{output_path}/captioned_piece_test_keyword_noLSTM.mid")
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
sp = midi.realtime.StreamPlayer(s)
sp.play()

pygame 2.5.1 (SDL 2.28.2, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
sp.stop